In [0]:
import os
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')
path = "/content/gdrive/My Drive/Colab Notebooks"
os.chdir(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from scipy.sparse import coo_matrix,dok_matrix
from scipy import sparse
import os
import math
import random
import numpy as np
from matplotlib import pyplot as plt
import datetime

In [0]:
start = datetime.datetime.now()
row=[]
col=[]
score=[]
max_item=0
with open("/content/gdrive/My Drive/RS/train.txt","r",encoding="utf8") as f:
    for line in f.readlines():
        if line=="\n" :
            break
        if "|" in line:
            user=int(line.split("|")[0])
            continue
        else:
            row.append(user)
            col.append(int(line.split()[0]))
            score.append(int(line.split()[1]))
            if int(line.split()[0])>max_item:
                max_item=int(line.split()[0])

new_usernum=user+1
print("user num:",new_usernum)
new_itemnum=max_item+1
print("item num:",new_itemnum)
print("phase1 end")

user num: 19835
item num: 624961
phase1 end


In [0]:
new_data=np.vstack((np.array(row),np.array(col)))
new_data=np.vstack((new_data,np.array(score))).T

random.shuffle(new_data)
print("data shape",new_data.shape)

train_num=int(0.9*len(new_data))
print("training_set number:",train_num)
traindata=new_data[:train_num,:]
testdata=new_data[train_num:,:]

Average_score=np.average(score)
print("parse2 end, Average score is",Average_score)

data shape (5001507, 3)
training_set number: 4501356
parse2 end, Average score is 49.54496554738402


In [0]:
def mult_1d(v1, v2):
	result = 0
	for i in range(len(v1)):
		result += v1[i] * v2[i]
	return result

def predict(aver, bu, bi, qi, px):
	res = aver + bu + bi + mult_1d(qi, px)
	if res < 0:
		res = 0
	if res > 100:
		res = 100
	return  res

def validate(testData, aver, bu, bi, qi, px):
	rmse = 0.0
	for i in range(len(testData)):
		user = testData[i][0]
		item = testData[i][1]
		score_hat = predict(aver, bu[user], bi[item], qi[item], px[user])
		score_gt = testData[i][2]
		rmse += (score_gt - score_hat) * (score_gt - score_hat)
	return math.sqrt(rmse / len(testData))

def test(aver,bu,bi,qi,pu):
	user=0
	item=0
	score=0
	with open("/content/gdrive/My Drive/RS/test.txt","r",encoding="utf8") as f1:
		with open("/content/gdrive/My Drive/RS/result.txt","w",encoding="utf8") as f2:
			for line in  f1.readlines():
				if line=="\n":
					break
				if "|" in line:
					f2.write(line)
					user=int(line.split("|")[0])
					continue
				else:
					item=int(line.split()[0])
					score=int(predict(aver, bu[user], bi[item], qi[item], pu[user]))
					f2.write(str(item)+" "+str(score)+"\n")

In [0]:
def SVD(testData, trainData, factorNum, Lr, reg):
	aver = Average_score
	userNum = new_usernum
	itemNum = new_itemnum
	# factorNum = 10
	# Lr = 0.95
	# reg = 0.2
	#28.70 10 0.95 0.2 30
	bi = np.zeros(itemNum)
	bu = np.zeros(userNum)
	qi = np.zeros((itemNum,factorNum))
	pu = np.zeros((userNum,factorNum))
	# qi = dok_matrix((itemNum,factorNum),dtype=np.float)
	# pu = dok_matrix((userNum,factorNum),dtype=np.float)
	last_Rmse = 10000.0
	Epochs = 30
	print("Training begins:")
	for epoch in range(Epochs):
		print("Epoch",epoch," begins")
		time1=datetime.datetime.now()
		for i in range(len(trainData)):
			if i%int(len(trainData)/5)==0:
				print(int(100*i/len(trainData)),"% of current epoch ends")
			user = trainData[i][0]
			item = trainData[i][1]
			score = trainData[i][2]		
			prediction = predict(aver, bu[user], bi[item], qi[item], pu[user])
			rxi = score - prediction
			bu[user] += Lr * (rxi - reg * bu[user])
			bi[item] += Lr * (rxi - reg * bi[item])	
			for k in range(factorNum):
				temp = pu[user][k]	
				pu[user][k] += Lr * (rxi * qi[item][k] - reg * pu[user][k])
				qi[item][k] += Lr * (rxi * temp - reg * qi[item][k])
		if Lr>0.1:
			Lr *= 0.9
		else:
			Lr = 0.05

		curr_Rmse = validate(testData, aver, bu, bi, qi, pu)
		print("RMSE of epoch",epoch,": ", curr_Rmse)
		if curr_Rmse >= last_Rmse:
			break
		else:
			last_Rmse = curr_Rmse
		time2=datetime.datetime.now()
		run_time=(time2-time1).seconds
		print("Epoch",epoch," running time:  ",int(run_time/60),"min",run_time%60,"s")
	print("Training ends. RMSE is",curr_Rmse)
	test(aver, bu, bi, qi, pu)

In [7]:
SVD(testdata,traindata,10,0.95,0.2)
end = datetime.datetime.now()
run_time=(end-start).seconds
print("Running time :  ",int(run_time/60),"min",run_time%60,"s")

Training begins:
Epoch 0  begins
0 % of current epoch ends
19 % of current epoch ends
39 % of current epoch ends
59 % of current epoch ends
79 % of current epoch ends
99 % of current epoch ends
RMSE of epoch 0 :  42.591104906557796
Epoch 0  running time:   5 min 7 s
Epoch 1  begins
0 % of current epoch ends
19 % of current epoch ends
39 % of current epoch ends
59 % of current epoch ends
79 % of current epoch ends
99 % of current epoch ends
RMSE of epoch 1 :  41.392931360493186
Epoch 1  running time:   5 min 0 s
Epoch 2  begins
0 % of current epoch ends
19 % of current epoch ends
39 % of current epoch ends
59 % of current epoch ends
79 % of current epoch ends
99 % of current epoch ends
RMSE of epoch 2 :  40.11921256185217
Epoch 2  running time:   4 min 50 s
Epoch 3  begins
0 % of current epoch ends
19 % of current epoch ends
39 % of current epoch ends
59 % of current epoch ends
79 % of current epoch ends
99 % of current epoch ends
RMSE of epoch 3 :  38.8727907904683
Epoch 3  running tim